# Advent of Code #


In [1]:
# set up the environment
import numpy as np

## Day 3: Binary Diagnostic ##
### Part 1 ###
The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```
Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [2]:
# Import the data into a text array - easier for getting positional data
diagnostics = np.genfromtxt("data/diagnostic.dat", dtype="U")
print(diagnostics[:10])
print(len(diagnostics))

['111010101100' '100001001100' '000111101100' '100100000000'
 '001001001110' '100110101011' '001001100101' '010000010110'
 '011011001001' '001001000101']
1000


In [3]:
# get the number of bits in the data
bits = len(diagnostics[0])
print(bits)

12


In [4]:
epsilon_str = ""
gamma_str = ""

for x in range(len(diagnostics[0])):
    gamma_val = 0
    for y in diagnostics:
        if y[x] == '1':
            gamma_val += 1
        else:
            gamma_val -= 1

# take advantage of the fact that the epsilon value == not(gamma value)
# (could be done later on the whole string but just as easy to build it now) 
    if gamma_val > 0:
        gamma_str += '1'
        epsilon_str +=  '0'
    else:
        gamma_str += '0'
        epsilon_str += '1'

# convert the base 2 strings to integers 
gamma_rate = int(gamma_str, 2)
epsilon_rate = int(epsilon_str, 2)

power_consumptions = gamma_rate * epsilon_rate

print("Gama rate: ", gamma_rate)
print("Epsilon rate: ", epsilon_rate)
print("Power Consumption: ", power_consumptions)


Gama rate:  3903
Epsilon rate:  192
Power Consumption:  749376


### Part Two ###

Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

    Keep only numbers selected by the bit criteria for the type of rating value for which you are searching.
        Discard numbers which do not match the bit criteria.
    If you only have one number left, stop; this is the rating value for which you are searching.
    Otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

For example, to determine the oxygen generator rating value using the same example diagnostic report from above:

    Start with all 12 numbers and consider only the first bit of each number. 
        There are more 1 bits (7) than 0 bits (5)
        keep only the 7 numbers with a 1 in the first position:
            11110, 10110, 10111, 10101, 11100, 10000, and 11001.
    Then, consider the second bit of the 7 remaining numbers:
        there are more 0 bits (4) than 1 bits (3)
        keep only the 4 numbers with a 0 in the second position:
            10110, 10111, 10101, and 10000.
    In the third position,
        three of the four numbers have a 1
        keep those three:
            10110, 10111, and 10101.
    In the fourth position,
        two of the three numbers have a 1
        keep those two:
            10110 and 10111.
    In the fifth position,
        there are an equal number of 0 bits and 1 bits (one each).
        to find the oxygen generator rating, keep the number with a 1 in that position:
            10111.
    As there is only one number left, stop;
        the oxygen generator rating is 10111, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

    Start again with all 12 numbers and consider only the first bit of each number.
        There are fewer 0 bits (5) than 1 bits (7)
        keep only the 5 numbers with a 0 in the first position:
            00100, 01111, 00111, 00010, and 01010.
    Then, consider the second bit of the 5 remaining numbers:
        there are fewer 1 bits (2) than 0 bits (3)
        keep only the 2 numbers with a 1 in the second position:
            01111 and 01010.
    In the third position, there are an equal number of 0 bits and 1 bits (one each).
        to find the CO2 scrubber rating, keep the number with a 0 in that position:
            01010.
    As there is only one number left, stop; 
        the CO2 scrubber rating is 01010, or 10 in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine?

In [5]:
# create the arrays that will be reduced to a single value
o2_array = diagnostics.copy()
co2_array = diagnostics.copy()

def reduce_array(in_array: np.ndarray, check_greater: bool, bit_pos: int):
    
    # Returns the reduced array that includes only the required roww

    # Attributes
    # in_array:  numpy:np.ndarray
    #    the array whose rows are to be reduced
    # check_greater: bool
    #    are we checking for a greater preponderance of bits (O2 generator) or fewer (CO2 scrubber)
    # bit_pos: int
    #    the bit position to check
    
    total_1_bits = 0
    total_0_bits = 0
    for x in in_array:
        if x[bit_pos] == '1':
            total_1_bits += 1
        else:
            total_0_bits += 1

    temp_array: np.ndarray = []
    for y in in_array:
        bit_value = y[bit_pos]
        if check_greater:
            want_1s = True if total_1_bits >= total_0_bits else False
            if want_1s and bit_value == '1':
                temp_array.append(y)
            else:
                if not want_1s and bit_value == '0':
                    temp_array.append(y)
        else:
            want_0s = True if total_0_bits <= total_1_bits else False
            if want_0s and bit_value =='0':
                temp_array.append(y)
            else:
                if not want_0s and bit_value =='1':
                    temp_array.append(y)

    return temp_array

number_of_bits = len(o2_array[0])

check_bit = 0
while check_bit < number_of_bits:
    if len(o2_array) > 1:
        # check the Oxygen Generator - pick the greatest bit value for retaining
        o2_array =  reduce_array(o2_array, True, check_bit)
    if len(co2_array) >1:
        # check the CO2 Scrubber - pick the least bit value for retaining
        co2_array = reduce_array(co2_array, False, check_bit)
    check_bit += 1

  
# convert the reduced arrays to integers
o2_generator_rating = int(o2_array[0], 2)
co2_scrubber_rating = int(co2_array[0], 2)

life_support_rating = o2_generator_rating * co2_scrubber_rating

print("O2 Generator rating: ", o2_generator_rating)
print("CO2 Scrubber rating: ", co2_scrubber_rating)

print("Life Support rating: ", life_support_rating)


O2 Generator rating:  3871
CO2 Scrubber rating:  613
Life Support rating:  2372923
